In [1]:
!pip uninstall -y unsloth_zoo
!pip install --upgrade "unsloth_zoo @ git+https://github.com/unslothai/unsloth-zoo.git"

!pip uninstall -y unsloth
!pip install --upgrade "unsloth @ git+https://github.com/unslothai/unsloth.git"

!pip uninstall -y xformers
!pip install --upgrade xformers
 
!pip install bitsandbytes tf-keras



Found existing installation: unsloth_zoo 2024.11.8
Uninstalling unsloth_zoo-2024.11.8:
  Successfully uninstalled unsloth_zoo-2024.11.8
  Cloning https://github.com/unslothai/unsloth-zoo.git to /tmp/pip-install-spm5o4hi/unsloth-zoo_d76d5ca9b21e46ee932bb3a9f8dc2fc2
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth-zoo.git /tmp/pip-install-spm5o4hi/unsloth-zoo_d76d5ca9b21e46ee932bb3a9f8dc2fc2
  Resolved https://github.com/unslothai/unsloth-zoo.git to commit f889b058c00fb06dac60a2c51d24ca441e505f3c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for unsloth_zoo: filename=unsloth_zoo-2024.11.8-py3-none-any.whl size=58648 sha256=2beaf3113a46c07dc5e47f6ac46e7bfbbaeef15cb3143010607b73f31b04e866
  Stored in directory: /tmp/pip-ephem-wheel-cache-jcxw_12e/wheels/3c/9c/61/d976beea5cf257b6d462acfad5f5d4edbb3cd18a564ce720ab
Successfully built unsloth_zoo
F

In [2]:
%%capture
!pip install unsloth "xformers==0.0.28.post2"
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [3]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 3072 
dtype = None 
load_in_4bit = True 

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

model = FastLanguageModel.get_peft_model(
    model,
    r = 16, 
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, 
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,  
    loftq_config = None, 
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2024-12-03 16:27:37.787217: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733243257.802159    5848 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733243257.806700    5848 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-03 16:27:37.824073: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2024.11.10: Fast Llama patching. Transformers:4.46.3.
   \\   /|    GPU: NVIDIA GeForce RTX 4090. Max memory: 23.546 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2024.11.10 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [4]:
import json
from datasets import load_dataset

alpaca_prompt = """Di bawah ini terdapat sebuah instruksi yang menjelaskan tugas, disertai dengan input yang memberikan konteks lebih lanjut. Model harus menganalisis dengan cermat input yang diberikan, yaitu sebuah transkrip pidato atau presentasi, dan kemudian menghasilkan respons terstruktur yang mencakup skor penilaian dan penjelasan komprehensif yang menyelesaikan tugas evaluasi. Penting: Anda WAJIB mematuhi format output yang telah ditentukan, termasuk penggunaan format JSON yang benar dan konsisten. TIDAK BOLEH menyertakan informasi tambahan di luar yang diminta atau mengulang kembali instruksi atau input dalam output. Pastikan setiap bagian output sesuai dengan perannya untuk memudahkan pemrosesan di sistem backend. Ketidakpatuhan terhadap format akan menyebabkan kegagalan dalam pemrosesan data.

### Instruction:
{}

### Input:
{}

### Explanation:
{}
"""

EOS_TOKEN = tokenizer.eos_token 

def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs = examples["input"]
    explanations = examples["explanation"]

    texts = []

    for instruction, input_text, explanation in zip(instructions, inputs, explanations):
        scores = explanation[0] 
        if "text" in scores:
            del scores["text"]  

        analysis_text = explanation[1]

        if isinstance(analysis_text, dict) and "text" in analysis_text:
            analysis_text = analysis_text["text"]

        formatted_analysis = {"text": analysis_text}
        formatted_explanation = f"[{json.dumps(scores, ensure_ascii=False)}, {json.dumps(formatted_analysis, ensure_ascii=False)}]"
        text = alpaca_prompt.format(instruction, input_text, formatted_explanation) + EOS_TOKEN
        texts.append(text)

    print("Pratinjau Dataset yang Diformat:")
    for i, text in enumerate(texts[:5]):
        print(f"Contoh {i + 1}:\n{text}\n")

    return {"text": texts}

dataset = load_dataset("json", data_files="final_dataset_kaizen.jsonl", split="train")
dataset = dataset.map(formatting_prompts_func, batched=True)
dataset.save_to_disk("formatted_dataset")

Saving the dataset (0/1 shards):   0%|          | 0/600 [00:00<?, ? examples/s]

In [5]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, 
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 20, 
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",
    ),
)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [6]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 600 | Num Epochs = 20
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 1,500
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,1.107400
2,1.117600
3,1.088300
4,1.111400
5,1.027500
6,0.956200
7,0.889300
8,0.792200
9,0.721300
10,0.646500


In [7]:
instruction = """Anda bertugas untuk mengevaluasi transkrip dari seseorang yang sedang berbicara dalam bentuk pidato atau presentasi. Evaluasi harus dilakukan dengan memperhatikan konteks dari input yang berupa transkrip untuk memberikan penilaian yang akurat terhadap kualitas berbicara. Ada empat aspek utama yang wajib Anda nilai: Kejelasan Berbicara, Penggunaan Diksi, Kelancaran dan Intonasi, serta Emosional dan Keterlibatan Audiens. Output Anda harus dalam bentuk JSON yang berisi array dengan dua elemen seperti contoh berikut:

[{"Kejelasan Berbicara": <skor>, "Penggunaan Diksi": <skor>, "Kelancaran dan Intonasi": <skor>, "Emosional dan Keterlibatan Audiens": <skor>}, {"text": "<teks analisis yang mendalam dan terstruktur>"}]


Penjelasan mengenai elemen dalam array JSON:

1. Elemen pertama adalah objek JSON yang memuat skor penilaian untuk keempat aspek berikut:
   {"Kejelasan Berbicara": 82, "Penggunaan Diksi": 78, "Kelancaran dan Intonasi": 72, "Emosional dan Keterlibatan Audiens": 80}
   
   Setiap aspek harus memiliki nilai skor dalam rentang 0-100, dengan 100 menunjukkan performa maksimal.

2. Elemen kedua adalah objek JSON yang memiliki properti "text" berisi analisis mendalam dalam format paragraf yang terstruktur. Teks analisis harus mencakup:
   - **Ulasan Umum**: Berikan ulasan yang mencakup kekuatan dan kelemahan transkrip secara keseluruhan, serta kaitkan dengan skor yang diberikan.
   - **Saran dan Solusi**: Sediakan saran praktis untuk memperbaiki kelemahan yang ditemukan, ditulis dalam format bulet poin untuk memudahkan pemahaman.
   - **Kesimpulan**: Ringkasan singkat tentang bagaimana pembicara dapat meningkatkan kemampuan berbicara mereka di masa depan.

Anda hanya perlu mengeluarkan jawaban output sesuai dengan format output yang saya minta saja, anda dilarang mengeluarkan kata kata tambahan di luar format output(json) karena dapat mengganggu pemrosesan data di backend nantinya. Anda hanya perlu mengeluarkan jawaban saja tanpa ada tambahan kata kata di awal.

Contoh format output yang benar untuk seluruh JSON:
[{"Kejelasan Berbicara": 82, "Penggunaan Diksi": 78, "Kelancaran dan Intonasi": 72, "Emosional dan Keterlibatan Audiens": 80}, {"text": "**Analisis**\n\nPidato ini disampaikan dengan struktur yang jelas dan logis, membuat audiens dapat dengan mudah mengikuti alur pemikiran pembicara. Namun, ada beberapa kelemahan yang perlu diperhatikan. Kejelasan berbicara (82) cukup baik, tetapi bisa ditingkatkan dengan latihan yang lebih intensif. Penggunaan diksi (78) cukup beragam, tetapi ada ruang untuk perbaikan dalam variasi kata. Kelancaran dan intonasi (72) masih dapat ditingkatkan untuk menghindari kesan monoton. Emosional dan keterlibatan audiens (80) cukup efektif, tetapi dapat diperkuat dengan ekspresi lebih mendalam.\n\n**Saran untuk Peningkatan**\n- Latih kelancaran berbicara untuk menghindari jeda yang tidak perlu.\n- Gunakan variasi kata yang lebih kaya untuk membuat pidato lebih menarik.\n- Latih ekspresi emosional untuk lebih melibatkan audiens.\n- Rekam diri sendiri dan tinjau untuk meningkatkan kesadaran akan intonasi dan emosi.\n\n**Kesimpulan**\n\nPidato ini memiliki dasar yang kuat, tetapi masih ada ruang untuk perbaikan. Dengan memperhatikan saran-saran di atas, pembicara dapat lebih efektif dan menarik."}]
"""
input = "Pada saat ini, kabai menyadari bahwa banyak masyarakat Indonesia mengencimsi makanan yang puan sehar. Banyak dari kita lebih memilih membeli makanan cepat saja di bandingkan dengan memasak makanan sendiri. Kita tahu bahwa kebiasaan mengencimsi makanan yang tidak sehar, dapat menyabakkan masalah kesehatan janggapan jan. Namun, kita juga menyadari bahwa meningkatkan kemampuan dari kesehatan yang berbeda. Kita juga bisa membangun kemampuan dari kesehatan yang muda, dan membangun kemampuan dari kesehatan yang muda. Namun, kita juga menyadari bahwa meningkatkan kualitas polah hidup dengan memasak makan malas sendiri juga sulit untuk dilakukan. Banyak orang ingin hidup sehat dengan memasak makanan sendiri, namun tidak tahu dari mana harus memulainya. Apalagi, kurangnya informasi mengenai cara masak serta pembelian pehan-pehan masakan yang biasanya terpisah bisa tempatnya menyabah bukan masak sendiri lebih sulit untuk dilakukan. Tentu, dengan memainkan dari permasalah yang ada, Masalahnya yang ada, langkah awalan berdilagukan adalah dengan meningkatkan kesejalan masyarakat, mengenai pentingnya makan makan nansihat, sehatan dan bapak negatif dari mengenciusi makan nansi saat saji Sarah Berlebihan. Namun, itu saja tidaklah cukup. Di era modern perbasih buk ini, banyak orang di Lat Milkiwahtu maupun kemampuan untuk memasak sendiri setiap harinya. Oleh karena itu, diberelukan solusi yang dapat membantu masyarakat, dalam mengatasi permasalahan ini dengan ceren efektif dan efisian. Salah satu solution dapat dilakukan adalah dengan memanfaatkan teknologi digital untuk memberikan access yang lebih mudah dan praktis dalam memasak makanan yang sihat dan enak. Dengan memanfaatkan teknologi digital, kami berharap solution kami talerkan dapat diakses oleh siapa saja dan kapan saja tanpa terbataast oleh waktu dan lokasi. Makadari itu, kami mengbanggantung sebuah ide yang dapat membantu sehulawisan masyarakat Indonesia terutama masyarakat dengan kesebukan yang padat untuk membuat masakan yang sehar dan enak dengan muda, melalui aplikasi held in all. Aplikasi held in all, menyadakan esat masakan sehar dengan video tutorial yang jelas dan muda diikuti. Aplikasi kami juga menyadakan jasa pembelian permasakan dengan informasi nilai kisi bahan secara lengkap. Aplikasi ini juga dilengkapi dengan artificial intelligence untuk meningkatkan duis air experience dan kemudahan dalam penggunaannya. Melalui pemanfaatan teknologi ini, pengguna diharapan dapat dengan muda memperoleh pilihan nilai kecil, yang tepat sesuai dengan perfranci dan keputuhannya. Healthi ini is a form of service bank. Dengan ini, kami berharap aplikasi healthi nama-dapat memberikan manfaat yang besar bagi bangan orang, dengan turut membantu menikatkan kualitas kesehatan hidup masyarakat Indonesia."

FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        instruction, 
        input,
        "",
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 2500)

<|begin_of_text|>Di bawah ini terdapat sebuah instruksi yang menjelaskan tugas, disertai dengan input yang memberikan konteks lebih lanjut. Model harus menganalisis dengan cermat input yang diberikan, yaitu sebuah transkrip pidato atau presentasi, dan kemudian menghasilkan respons terstruktur yang mencakup skor penilaian dan penjelasan komprehensif yang menyelesaikan tugas evaluasi. Penting: Anda WAJIB mematuhi format output yang telah ditentukan, termasuk penggunaan format JSON yang benar dan konsisten. TIDAK BOLEH menyertakan informasi tambahan di luar yang diminta atau mengulang kembali instruksi atau input dalam output. Pastikan setiap bagian output sesuai dengan perannya untuk memudahkan pemrosesan di sistem backend. Ketidakpatuhan terhadap format akan menyebabkan kegagalan dalam pemrosesan data.

### Instruction:
Anda bertugas untuk mengevaluasi transkrip dari seseorang yang sedang berbicara dalam bentuk pidato atau presentasi. Evaluasi harus dilakukan dengan memperhatikan kontek

In [8]:
model.save_pretrained_merged("modellorav10", tokenizer, save_method = "lora",)

Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... Done.
